# ++ Carregar dados históricos da Bovespa em Dataframe para análise ++

In [1]:
import time
time.asctime( time.localtime(time.time()) )

'Thu May 25 15:47:25 2023'

In [2]:
import pandas as pd
from datetime import date, datetime, timedelta
# pd.__version__

In [3]:
from IPython.display import FileLink, FileLinks
FileLink('Manipulação CSV BB.ipynb')

C:\Users\F0435280\OneDrive - Banco do Brasil S.A\Área de Trabalho\Alexandre\StockManager-main\Manipulação CSV BB.ipynb

In [4]:
# Setando opção do Pandas para mostrar todas as linhas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Setando separador de milhar
pd.options.display.float_format = '{:,.2f}'.format

In [5]:
from pacote.functions import *

In [6]:
# LISTA_ARQUIVOS = ['A2023', 'M112022', 'M122022']
LISTA_ARQUIVOS = ['A2023']

In [7]:
df = carrega_dados(LISTA_ARQUIVOS)

In [8]:
from IPython.display import Markdown, display

# Monday is 0 and Sunday is 6.
dia_semana  = datetime.today().weekday()
ajuste_dias = -3 if dia_semana == 0 else -1
dt_esperado_ult = (datetime.today() + timedelta(days=ajuste_dias)).strftime('%Y-%m-%d')
dt_max = str(df["dtPregao"].max()).split()[0]

if dt_esperado_ult == dt_max:
    font_color="alert-success"
    a = "Atualizado!"
else:
    font_color="alert-danger"
    a = "Desatualizado!!! Última data " + str(dt_max)
def printmd(string):
    display(Markdown(string))

In [9]:
# df_n_dias, vol, col_pc, pc_min, avg_vr_fech
d25 = monta_tabela(monta_df_periodos(df, 25), 6000000, '1.0%', 22, 2)
d20 = monta_tabela(monta_df_periodos(df, 20), 6000000, '1.0%', 19, 2)
d15 = monta_tabela(monta_df_periodos(df, 15), 6000000, '1.0%', 15, 2)
d10 = monta_tabela(monta_df_periodos(df, 10), 6000000, '1.0%', 10, 2)
d05 = monta_tabela(monta_df_periodos(df, 5), 6000000, '1.0%', 5, 2)
merge1 = pd.merge(d25['cdAcao'], d20['cdAcao'], how ='inner', on =['cdAcao'])
merge2 = pd.merge(merge1, d15['cdAcao'], how ='inner', on =['cdAcao'])
merge3 = pd.merge(merge2, d10['cdAcao'], how ='inner', on =['cdAcao'])
merge_final = pd.merge(merge3, d05['cdAcao'], how ='inner', on =['cdAcao'])

In [10]:
printmd(f'**<div class={font_color}>Arquivo {a}</div>**')

**<div class=alert-success>Arquivo Atualizado!</div>**

### 25 dias, order by 1%

In [11]:
d25

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
342,RCSL3,23,23,23,22,20,17,"6,225,052.04",2.39,-2.57
378,SMFT3,23,23,21,21,14,10,"49,012,757.96",15.79,0.02
148,DASA3,23,23,21,20,17,15,"7,153,279.64",9.43,0.05
219,HBSA3,23,23,19,17,11,9,"12,322,763.84",2.68,0.29
338,RAIZ4,22,22,22,17,13,11,"78,186,975.44",3.28,0.14
253,LIGT3,23,22,21,21,21,21,"61,916,224.20",3.57,-0.74
275,MLAS3,23,22,21,19,18,18,"8,879,496.48",2.05,0.02
280,MOVI3,24,22,19,17,14,13,"42,573,363.40",9.51,-0.04
341,RAPT4,24,22,18,11,8,6,"25,891,029.76",9.13,-0.15


### 20 dias, order by 1%

In [12]:
d20

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
147,DASA3,20,20,19,18,15,14,"7,606,344.55",9.99,0.01
377,SMFT3,20,20,18,18,11,8,"55,759,845.75",16.17,0.26
218,HBSA3,20,20,16,14,10,8,"12,923,319.75",2.74,0.28
398,TEND3,19,19,19,17,15,15,"32,597,971.65",6.90,0.80
33,AZUL4,19,19,18,16,14,12,"217,564,642.95",12.53,0.41
279,MOVI3,20,19,16,14,12,11,"44,738,374.70",9.91,-0.04
340,RAPT4,20,19,16,10,7,5,"26,329,680.00",9.43,-0.19
289,MYPK3,19,19,14,8,7,5,"23,333,760.50",11.66,-0.04


### 15 dias, order by 1%

In [13]:
d15

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
145,DASA3,15,15,14,14,12,11,"6,865,304.47",10.64,0.18
144,CYRE3,15,15,14,13,8,4,"90,630,307.20",16.66,0.03
374,SMFT3,15,15,13,13,7,4,"66,040,566.87",16.52,0.31
419,VBBR3,15,15,13,12,10,8,"248,444,070.47",15.17,0.13
348,ROMI3,15,15,12,10,8,7,"8,329,254.87",14.95,0.48
337,RAPT4,15,15,12,7,6,4,"28,751,297.53",9.81,-0.35
216,HBSA3,15,15,11,9,7,6,"14,081,243.80",2.79,0.19


### 10 dias, order by 1%

In [14]:
d10

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
330,RAIZ4,10,10,10,6,5,3,"104,631,042.70",3.44,0.32
143,DASA3,10,10,9,9,7,6,"6,668,887.60",10.93,-0.29
415,VBBR3,10,10,9,8,7,5,"285,084,066.60",15.77,-0.27
142,CYRE3,10,10,9,8,4,1,"94,964,420.50",17.00,-0.07
403,TUPY3,10,10,9,7,5,3,"19,234,105.00",24.25,0.77
212,HBSA3,10,10,8,8,6,5,"15,898,526.60",2.86,0.32
278,MTRE3,10,10,8,8,5,5,"6,733,055.90",4.53,0.08
370,SMFT3,10,10,8,8,4,3,"76,649,590.60",16.86,0.40
283,MYPK3,10,10,8,3,3,2,"20,327,844.50",11.99,-0.02
344,ROMI3,10,10,7,6,6,5,"8,343,641.40",15.20,0.23


### 5 dias, order by 1%

In [15]:
d05

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech,AvgPcAbert
238,LJQQ3,5,5,5,5,5,4,"30,385,251.80",4.24,0.56
137,DASA3,5,5,5,5,4,4,"6,214,006.40",11.08,0.07
400,VBBR3,5,5,5,5,4,3,"308,205,739.20",16.76,-0.24
136,CYRE3,5,5,5,5,4,1,"96,614,809.00",17.35,-0.16
417,ZAMP3,5,5,5,5,3,3,"9,722,023.40",3.88,0.32
387,TTEN3,5,5,5,5,3,1,"15,666,254.00",11.99,0.10
225,JHSF3,5,5,5,4,4,3,"18,452,223.00",4.38,-0.18
295,PETZ3,5,5,5,4,4,3,"68,939,866.00",7.43,-0.00
330,ROMI3,5,5,5,4,4,3,"9,393,922.00",15.74,0.18
401,VIIA3,5,5,5,4,4,3,"127,165,840.80",2.23,-0.09


In [16]:
printmd(f'**<div class={font_color}>Status {a}</div>**')

**<div class=alert-success>Status Atualizado!</div>**

### Consulta ação específica

In [17]:
consulta_acao(df, "AZUL4")

,cdAcao,dtPregao,vrFech,vrVolume,vrMaxDia,vrMinDia,vrAbert,pcVar,pcMaxDia,pcMinDia,pcAbert,ic05,ic10,ic15,ic20,ic25,ic30
36332,AZUL4,2023-05-24,14.69,"169,615,312",14.95,14.22,14.74,-1.61,0.13,-4.76,-1.27,,,,,,
35672,AZUL4,2023-05-23,14.93,"244,804,708",15.81,14.84,15.44,-3.05,2.66,-3.64,0.26,1,1,1,1,1,
35431,AZUL4,2023-05-22,15.40,"417,170,192",15.91,14.51,14.58,9.38,13.00,3.05,3.55,1,1,1,1,1,1
34954,AZUL4,2023-05-19,14.08,"167,903,779",14.73,13.97,14.45,-2.90,1.59,-3.66,-0.34,1,1,1,,,
34586,AZUL4,2023-05-18,14.50,"236,413,145",14.62,14.08,14.19,2.11,2.96,-0.85,-0.07,1,1,1,1,1,
34439,AZUL4,2023-05-17,14.20,"417,143,374",14.40,12.53,12.67,14.89,16.50,1.38,2.51,1,1,1,1,1,1
33867,AZUL4,2023-05-16,12.36,"238,089,640",13.44,12.16,12.66,-3.13,5.33,-4.70,-0.78,1,1,1,1,1,1
33429,AZUL4,2023-05-15,12.76,"396,792,656",13.09,11.35,13.06,-0.78,1.79,-11.74,1.56,1,1,1,,,
33289,AZUL4,2023-05-12,12.86,"230,781,396",13.14,12.62,12.97,-0.85,1.31,-2.70,,1,1,,,,
32967,AZUL4,2023-05-11,12.97,"231,308,219",13.03,12.25,12.31,5.53,6.02,-0.33,0.16,1,1,1,1,1,1


### Ações presentes em todos os períodos

In [18]:
merge_final

,cdAcao
0,SMFT3
1,DASA3
2,HBSA3
3,RAPT4


### ======================================== TESTES ========================================

### Comparar Maiores Lucros

In [19]:
df_1 = monta_lucro_periodo(df, 10, 0, True)
df_2 = monta_lucro_periodo(df, 10, 1, True)

2023-05-11 >> 2023-05-24
2023-05-10 >> 2023-05-23


In [20]:
df_final = df_1.copy()
df_final['posicao ant'], df_final['mudanca'] = [
    df_2['posicao']
    , df_2['posicao'] - df_1['posicao']
]
df_final['posicao ant'] = df_final['posicao ant'].fillna(0).astype('int')
df_final['mudanca'] = df_final['mudanca'].fillna(0).astype('int')

In [21]:
df_final.sort_values(["mudanca"], ascending=False)

,dtInicio,dtFim,vrInicio,vrFim,pcPeriodo,avgVol,posicao,posicao ant,mudanca
cdAcao,,,,,,,,,
ABCB4,2023-05-11,NaT,16.50,NaN,NaN,"20,303,131.00",1,1,0
RAPT4,2023-05-11,NaT,9.89,NaN,NaN,"31,742,255.11",110,110,0
NTCO3,2023-05-11,NaT,12.92,NaN,NaN,"206,177,949.89",94,94,0
ODPV3,2023-05-11,NaT,11.07,NaN,NaN,"23,717,633.22",95,95,0
ONCO3,2023-05-11,NaT,10.12,NaN,NaN,"27,956,634.33",96,96,0
ORVR3,2023-05-11,NaT,37.50,NaN,NaN,"20,385,537.11",97,97,0
PCAR3,2023-05-11,NaT,16.24,NaN,NaN,"37,451,854.89",98,98,0
PETR3,2023-05-11,NaT,28.15,NaN,NaN,"552,482,969.11",99,99,0
PETR4,2023-05-11,NaT,25.43,NaN,NaN,"2,260,795,120.11",100,100,0


In [22]:
df_final.sort_values(["mudanca"], ascending=False).loc[df_final['posicao ant'] <= 10]

,dtInicio,dtFim,vrInicio,vrFim,pcPeriodo,avgVol,posicao,posicao ant,mudanca
cdAcao,,,,,,,,,
ABCB4,2023-05-11,NaT,16.50,NaN,NaN,"20,303,131.00",1,1,0
ABEV3,2023-05-11,NaT,14.50,NaN,NaN,"282,595,523.33",2,2,0
ARML3,2023-05-11,NaT,11.41,NaN,NaN,"8,428,866.33",9,9,0
AESB3,2023-05-11,NaT,12.49,NaN,NaN,"25,132,034.22",3,3,0
AGRO3,2023-05-11,NaT,22.75,NaN,NaN,"8,960,856.78",4,4,0
ALPA4,2023-05-11,NaT,9.01,NaN,NaN,"114,240,554.11",5,5,0
ALSO3,2023-05-11,NaT,19.40,NaN,NaN,"120,897,125.89",6,6,0
ALUP11,2023-05-11,NaT,28.09,NaN,NaN,"25,092,749.67",7,7,0
AMBP3,2023-05-11,NaT,21.56,NaN,NaN,"37,795,728.44",8,8,0


In [23]:
df_1

,dtInicio,dtFim,vrInicio,vrFim,pcPeriodo,avgVol,posicao
cdAcao,,,,,,,
ABCB4,2023-05-11,NaT,16.50,NaN,NaN,"20,303,131.00",1
ABEV3,2023-05-11,NaT,14.50,NaN,NaN,"282,595,523.33",2
AESB3,2023-05-11,NaT,12.49,NaN,NaN,"25,132,034.22",3
AGRO3,2023-05-11,NaT,22.75,NaN,NaN,"8,960,856.78",4
ALPA4,2023-05-11,NaT,9.01,NaN,NaN,"114,240,554.11",5
ALSO3,2023-05-11,NaT,19.40,NaN,NaN,"120,897,125.89",6
ALUP11,2023-05-11,NaT,28.09,NaN,NaN,"25,092,749.67",7
AMBP3,2023-05-11,NaT,21.56,NaN,NaN,"37,795,728.44",8
ARML3,2023-05-11,NaT,11.41,NaN,NaN,"8,428,866.33",9


In [24]:
df_2

,dtInicio,dtFim,vrInicio,vrFim,pcPeriodo,avgVol,posicao
cdAcao,,,,,,,
ABCB4,2023-05-10,NaT,16.38,NaN,NaN,"20,214,362.56",1
ABEV3,2023-05-10,NaT,14.40,NaN,NaN,"313,540,784.11",2
AESB3,2023-05-10,NaT,12.61,NaN,NaN,"26,408,565.33",3
AGRO3,2023-05-10,NaT,22.66,NaN,NaN,"10,402,779.22",4
ALPA4,2023-05-10,NaT,9.33,NaN,NaN,"104,387,948.67",5
ALSO3,2023-05-10,NaT,19.39,NaN,NaN,"117,337,173.22",6
ALUP11,2023-05-10,NaT,27.96,NaN,NaN,"24,951,440.89",7
AMBP3,2023-05-10,NaT,20.75,NaN,NaN,"39,906,142.11",8
ARML3,2023-05-10,NaT,12.00,NaN,NaN,"9,675,494.67",9


In [25]:
# dfDtMin = df5d.loc[(df["dtPregao"] == dt_min_5d) & (df["cdAcao"] == 'PTBL3')]

df5d  = busca_periodos(df, 5)
df25d = busca_periodos(df, 25)

dt_min_5d  = df5d["dtPregao"].min()
dt_min_25d = df25d["dtPregao"].min()
dt_max     = df5d["dtPregao"].max()
dfDtMin5d  = df5d.loc[(df5d["dtPregao"] == dt_min_5d)].set_index(["cdAcao"])
dfDtMin25d = df25d.loc[(df25d["dtPregao"] == dt_min_25d)].set_index(["cdAcao"])
dfDtMax    = df5d.loc[(df5d["dtPregao"] == dt_max)].set_index(["cdAcao"])
dfAvgVol5d  = busca_media(df5d, "vrVolume", "vol").set_index(["cdAcao"])
dfAvgVol25d = busca_media(df25d, "vrVolume", "vol").set_index(["cdAcao"])

In [26]:
 dfPc5d = pd.DataFrame({
         "dtInicio":dfDtMin5d["dtPregao"] ,"dtFim":dfDtMax["dtPregao"]
        ,"vrInicio":dfDtMin5d["vrFech"]   ,"vrFim":dfDtMax["vrFech"]
        ,"pcPeriodo":((dfDtMax["vrFech"] - dfDtMin5d["vrFech"]) / dfDtMin5d["vrFech"]) * 100
        ,"avgVol":dfAvgVol5d["vol"]
    })

In [27]:
 dfPc25d = pd.DataFrame({
         "dtInicio":dfDtMin25d["dtPregao"] ,"dtFim":dfDtMax["dtPregao"]
        ,"vrInicio":dfDtMin25d["vrFech"]   ,"vrFim":dfDtMax["vrFech"]
        ,"pcPeriodo":((dfDtMax["vrFech"] - dfDtMin25d["vrFech"]) / dfDtMin25d["vrFech"]) * 100
        ,"avgVol":dfAvgVol25d["vol"]
    })

In [28]:
dfPc25d.sort_values(["pcPeriodo"], ascending=False).loc[(dfPc25d["avgVol"] > 6000000)]

,dtInicio,dtFim,vrInicio,vrFim,pcPeriodo,avgVol
cdAcao,,,,,,
LIGT3,2023-04-18,2023-05-24,2.13,4.61,116.43,"61,916,224.20"
YDUQ3,2023-04-18,2023-05-24,7.92,13.37,68.81,"63,848,401.20"
HAPV3,2023-04-18,2023-05-24,2.40,3.55,47.92,"355,586,188.24"
MDIA3,2023-04-18,2023-05-24,26.46,38.60,45.88,"53,006,885.80"
CEAB3,2023-04-18,2023-05-24,2.91,4.24,45.70,"17,672,081.00"
ALPA4,2023-04-18,2023-05-24,7.24,10.25,41.57,"94,459,654.32"
IRBR3,2023-04-18,2023-05-24,25.08,35.05,39.75,"116,017,916.32"
DASA3,2023-04-18,2023-05-24,8.02,11.06,37.91,"7,153,279.64"
LWSA3,2023-04-18,2023-05-24,5.39,7.36,36.55,"57,804,199.84"


In [29]:
dfPc5d.sort_values(["pcPeriodo"], ascending=False).loc[(dfPc5d["avgVol"] > 6000000)]

,dtInicio,dtFim,vrInicio,vrFim,pcPeriodo,avgVol
cdAcao,,,,,,
ALPA4,2023-05-18,2023-05-24,8.88,10.25,15.43,"171,964,046.20"
MODL3,2023-05-18,2023-05-24,2.21,2.39,8.14,"11,107,313.40"
MILS3,2023-05-18,2023-05-24,9.93,10.69,7.65,"15,204,683.80"
ZAMP3,2023-05-18,2023-05-24,3.84,4.11,7.03,"9,722,023.40"
ONCO3,2023-05-18,2023-05-24,10.21,10.84,6.17,"27,971,269.00"
VBBR3,2023-05-18,2023-05-24,16.20,17.16,5.93,"308,205,739.20"
LIGT3,2023-05-18,2023-05-24,4.39,4.61,5.01,"48,350,767.80"
PTBL3,2023-05-18,2023-05-24,5.93,6.20,4.55,"6,756,893.40"
ANIM3,2023-05-18,2023-05-24,2.74,2.86,4.38,"33,992,053.20"


In [30]:
dfPc25d['pcPeriodo']

cdAcao
AALR3      0.40
ABCB4      2.53
ABEV3      0.21
AERI3     -1.32
AESB3     17.60
AFLT3       NaN
AGRO3     -6.96
AGXY3    -16.85
AHEB3       NaN
AHEB5       NaN
AHEB6       NaN
ALLD3     22.46
ALPA3     34.03
ALPA4     41.57
ALPK3    102.33
ALSO3     13.52
ALUP11     7.04
ALUP3      5.86
ALUP4      5.23
AMAR3     -1.52
AMBP3    -10.54
AMER3     -4.35
ANIM3     19.67
APER3     -0.11
ARML3     25.02
ARZZ3      3.83
ASAI3    -14.82
ATMP3     -6.80
ATOM3     -8.07
AURE3     -7.40
AVLL3      5.45
AZEV3     -0.72
AZEV4      1.57
AZUL4     32.10
B3SA3      8.16
BAHI3       NaN
BALM3       NaN
BALM4       NaN
BAUH4       NaN
BAZA3     -3.57
BBAS3      2.36
BBDC3      9.25
BBDC4     14.08
BBSE3     -9.36
BDLL3       NaN
BDLL4       NaN
BEEF3      6.81
BEES3     11.67
BEES4     18.27
BGIP3       NaN
BGIP4     21.90
BIOM3     34.99
BLAU3     -4.39
BMEB3     -4.46
BMEB4     -0.89
BMGB4      7.25
BMIN3       NaN
BMIN4     -1.25
BMKS3       NaN
BMOB3     -3.19
BNBR3       NaN
BOAS3     -0.39
B

In [31]:
df_temp = monta_lucro_periodo(df, 25, 0, True)
# monta_lucro_periodo(df, 25, 0)

2023-04-18 >> 2023-05-24


In [32]:
df_temp

,dtInicio,dtFim,vrInicio,vrFim,pcPeriodo,avgVol,posicao
cdAcao,,,,,,,
ABCB4,2023-04-18,NaT,16.62,NaN,NaN,"15,237,182.67",1
ABEV3,2023-04-18,NaT,14.63,NaN,NaN,"346,719,793.62",2
AESB3,2023-04-18,NaT,10.51,NaN,NaN,"41,987,835.17",3
AGRO3,2023-04-18,NaT,25.00,NaN,NaN,"9,469,372.79",4
ALPA4,2023-04-18,NaT,7.24,NaN,NaN,"92,860,651.58",5
ALSO3,2023-04-18,NaT,18.34,NaN,NaN,"105,951,885.71",6
ALUP11,2023-04-18,NaT,26.84,NaN,NaN,"30,414,469.17",7
AMBP3,2023-04-18,NaT,21.91,NaN,NaN,"28,810,755.33",8
ARML3,2023-04-18,NaT,12.31,NaN,NaN,"6,431,435.71",9


In [33]:
df_n_dias = busca_periodos(df, 26)

dias_ant = 2
i = 0
while i <= dias_ant:
    dt_max  = df_n_dias["dtPregao"].max()
    df_n_dias = df_n_dias.loc[df_n_dias["dtPregao"] != dt_max]
    i += 1
    
dt_max

Timestamp('2023-05-22 00:00:00')

In [34]:
### 25 dias, Order by 0.5% # df05.loc[(df05["AvgVol"] > 6000000) & (df05["0.5%"] >= 23) & (df05["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)
### 20 dias, Order by 0.5% # df05_20d.loc[(df05_20d["AvgVol"] > 6000000) & (df05_20d["0.5%"] >= 19) & (df05_20d["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)
### 15 dias, Order by 0.5% # df05_15d.loc[(df05_15d["AvgVol"] > 6000000) & (df05_15d["0.5%"] >= 15) & (df05_15d["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)

## Definindo valores para compra

In [35]:
# Valor disponível para cmp / 1000
# Valor da ação no leilão
vr_corretagem = 9.8
emolumentos = 0.000325
acao = 'LIGT3'
vr_disp = 110
vr_acao_leilao = 1148 / 100
vr_qtd = round((vr_disp * 1000) / (vr_acao_leilao * 100)) * 100

In [36]:
vr_cmp = round(vr_acao_leilao * 1.014, 2)

vr_tot_cmp = vr_acao_leilao * vr_qtd
corret_cmp = vr_corretagem + (vr_tot_cmp * emolumentos)
vr_tot_liq_cmp = vr_tot_cmp + corret_cmp

vr_acao_vnd_10 = float('{0:.2f}'.format(vr_acao_leilao * 1.0108 + 0.01))
vr_tot_vnd_10 = vr_acao_vnd_10 * vr_qtd
corret_vnd_10 = vr_corretagem + (vr_tot_vnd_10 * emolumentos)
vr_tot_liq_vnd_10 = vr_tot_vnd_10 - corret_vnd_10

vr_acao_vnd_15 = float('{0:.2f}'.format(vr_acao_leilao * 1.0158 + 0.01))
vr_tot_vnd_15 = vr_acao_vnd_15 * vr_qtd
corret_vnd_15 = vr_corretagem + (vr_tot_vnd_15 * emolumentos)
vr_tot_liq_vnd_15 = vr_tot_vnd_15 - corret_vnd_15

In [37]:
d = {'Compra':     ['Quantidade', 'Valor Leilão', 'Total', 'Total líquido', 'Corretagem Compra'],
     ' ':          [vr_qtd, vr_acao_leilao, vr_tot_cmp, vr_tot_liq_cmp, corret_cmp],
     'Venda 1,0%': ['Valor compra', 'Valor', 'Total', 'Total líquido', 'Corretagem Venda'],
     '1,0%':       [vr_cmp, vr_acao_vnd_10, vr_tot_vnd_10, vr_tot_liq_vnd_10, corret_vnd_10],
     'Venda 1,5%': [' ', 'Valor', 'Total', 'Total líquido', 'Corretagem Venda'],
     '1,5%':       [' ', vr_acao_vnd_15, vr_tot_vnd_15, vr_tot_liq_vnd_15, corret_vnd_15]
    }
df_calc = pd.DataFrame(data=d)
df_calc

,Compra,,"Venda 1,0%","1,0%","Venda 1,5%","1,5%"
0,Quantidade,"9,600.00",Valor compra,11.64,,
1,Valor Leilão,11.48,Valor,11.61,Valor,11.67
2,Total,"110,208.00",Total,"111,456.00",Total,"112,032.00"
3,Total líquido,"110,253.62",Total líquido,"111,409.98",Total líquido,"111,985.79"
4,Corretagem Compra,45.62,Corretagem Venda,46.02,Corretagem Venda,46.21


In [38]:
d2 = {'Index':      ['Quantidade', 'Valor', 'Total', 'Total líquido', 'Corretagem'],
      'Compra':     [vr_qtd, vr_cmp, vr_tot_cmp, vr_tot_liq_cmp, corret_cmp],
      'Venda 1.0%': [vr_qtd, vr_acao_vnd_10, vr_tot_vnd_10, vr_tot_liq_vnd_10, corret_vnd_10],
      'Venda 1.5%': [vr_qtd, vr_acao_vnd_15, vr_tot_vnd_15, vr_tot_liq_vnd_15, corret_vnd_15]
     }
df_calc2 = pd.DataFrame(data=d2)
df_calc2.set_index('Index')
df_calc2

,Index,Compra,Venda 1.0%,Venda 1.5%
0,Quantidade,"9,600.00","9,600.00","9,600.00"
1,Valor,11.64,11.61,11.67
2,Total,"110,208.00","111,456.00","112,032.00"
3,Total líquido,"110,253.62","111,409.98","111,985.79"
4,Corretagem,45.62,46.02,46.21


In [39]:
time.asctime( time.localtime(time.time()) )

'Thu May 25 15:47:34 2023'